In [14]:
import torch
import torch.nn as nn
import torch_pruning as tp
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import os

# Define the LSTM model class as provided

In [16]:
class LSTMmodel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, output_size: int, dropout_rate: float = 0.5):
        super(LSTMmodel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.model_type = 'LSTM'

        # Define LSTM layer with dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)

        # Define a fully connected layer to map LSTM output to the target
        self.fc = nn.Linear(hidden_size, output_size)

        # Define a dropout layer before the fully connected layer
        self.dropout = nn.Dropout(dropout_rate)

        # Initialize weights
        self.init_weights()

    def init_weights(self) -> None:
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out

# Dataset class for the Appliances Energy Prediction dataset

In [17]:
class EnergyDataset(Dataset):
    def __init__(self, data, seq_len, feature_cols, target_col='Appliances'):
        self.seq_len = seq_len
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.scaler = MinMaxScaler()
        # Scale all data together
        self.data_scaled = self.scaler.fit_transform(data[feature_cols + [target_col]])

    def __len__(self):
        return len(self.data_scaled) - self.seq_len

    def __getitem__(self, idx):
        sequence = self.data_scaled[idx:idx + self.seq_len, :-1]  # Exclude target from sequence
        label = self.data_scaled[idx + self.seq_len, -1]  # Target is the last column
        return torch.tensor(sequence, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# Function to load and prepare data

In [18]:
def get_data_loaders(data_path, seq_len=24, batch_size=32, feature_cols=None, target_col='Appliances'):
    # Load the dataset
    data = pd.read_csv(data_path)

    # Default feature columns (all except 'date', 'Appliances', 'lights')
    if feature_cols is None:
        feature_cols = [
            'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5',
            'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
            'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint'
        ]

    # Select relevant columns
    data = data[feature_cols + [target_col]]

    # Drop rows with missing values
    data = data.dropna()

    # Create dataset
    dataset = EnergyDataset(data, seq_len, feature_cols, target_col)

    # Split into train and test sets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Training function

In [19]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

# Pruning function with structured pruning

In [20]:
def prune_model(model, example_input, train_loader, strategy, pruning_ratio=0.5, iterative_steps=5):
    pruner = strategy['pruner'](
        model,
        example_input,
        importance=strategy['importance'],
        iterative_steps=iterative_steps,
        ch_sparsity=pruning_ratio,
        root_module_types=[nn.LSTM],
        ignored_layers=[model.fc],
    )

    for i in range(iterative_steps):
        if isinstance(strategy['importance'], tp.importance.TaylorImportance):
            model.train()
            model.zero_grad()
            inputs, labels = next(iter(train_loader))
            inputs, labels = inputs.to(example_input.device), labels.to(example_input.device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
        pruner.step()
    return model

# Evaluation function (MSE for regression)

In [21]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    predictions, targets = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
            total_mae += torch.mean(torch.abs(outputs.squeeze() - labels)).item()
            predictions.extend(outputs.squeeze().cpu().numpy())
            targets.extend(labels.cpu().numpy())

    mse = total_loss / len(test_loader)
    mae = total_mae / len(test_loader)
    r2 = 1 - (sum((t - p) ** 2 for t, p in zip(targets, predictions)) / sum((t - np.mean(targets)) ** 2 for t in targets))

    print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
    return mse, mae, r2

# Save model

In [22]:
def save_model(model, path, example_input):
    torch.save(model.state_dict(), path)
    onnx_path = path.replace('.pth', '.onnx')
    torch.onnx.export(
        model,
        example_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f"Model saved to {path} and {onnx_path}")

# Main function

In [26]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Configuration dictionary
    config = {
        'strategies': {
            'magnitude': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.MagnitudeImportance(p=2),
            },
            'bn_scale': {
                'pruner': tp.pruner.BNScalePruner,
                'importance': tp.importance.BNScaleImportance(),
            },
            'group_norm': {
                'pruner': tp.pruner.GroupNormPruner,
                'importance': tp.importance.GroupMagnitudeImportance(p=1),
            },
            'random': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.RandomImportance(),
            },
            'Taylor': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.TaylorImportance()
            },
            'Hessian': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.GroupHessianImportance()
            },
            'lamp': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.LAMPImportance(p=2)
            },
            'geometry': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.FPGMImportance()
            },
        },
        'target_macs_sparsity': 0.5,
        'train_epochs': 10,
        'fine_tune_epochs': 20,
        'data_dir': './data',
        'output_dir': './output/strategies',
        'iterative_steps': 5,
    }

    # Initialize model parameters
    input_size = 24  # Number of features
    hidden_size = 64
    num_layers = 1
    output_size = 1
    dropout_rate = 0.5
    seq_len = 24
    batch_size = 32
    data_path = os.path.join(config['data_dir'], 'energydata_complete.csv')

    # Initialize model, criterion, and optimizer
    model = LSTMmodel(input_size, hidden_size, num_layers, output_size, dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Load data
    train_loader, test_loader = get_data_loaders(data_path, seq_len=seq_len, batch_size=batch_size)

    # Train initial model
    print("Training initial model...")
    train_model(model, train_loader, criterion, optimizer, device, num_epochs=config['train_epochs'])

    # Evaluate initial model
    print("Evaluating initial model...")
    mse, mae, r2 = evaluate_model(model, test_loader, criterion, device)

    # Save initial model
    initial_model_path = os.path.join(config['output_dir'], 'lstm_initial.pth')
    example_input = torch.randn(1, seq_len, input_size).to(device)
    save_model(model, initial_model_path, example_input)

    # Pruning loop for each strategy
    for strategy_name, strategy in config['strategies'].items():
        print(f"\nPruning with strategy: {strategy_name}")
        model_copy = LSTMmodel(input_size, hidden_size, num_layers, output_size, dropout_rate).to(device)
        model_copy.load_state_dict(torch.load(initial_model_path))

        # Prune the model
        pruned_model = prune_model(model_copy, example_input, train_loader, strategy, iterative_steps=config['iterative_steps'])

        # Fine-tune the pruned model
        optimizer = torch.optim.Adam(pruned_model.parameters(), lr=0.001)
        train_model(pruned_model, train_loader, criterion, optimizer, device, num_epochs=config['fine_tune_epochs'])

        # Evaluate pruned model
        print(f"Evaluating pruned model with {strategy_name}...")
        mse, mae, r2 = evaluate_model(pruned_model, test_loader, criterion, device)

        # Save pruned and fine-tuned model
        pruned_model_path = os.path.join(config['output_dir'], f'lstm_{strategy_name}_pruned.pth')
        save_model(pruned_model, pruned_model_path, example_input)

    print("All pruning strategies completed.")

# Run the main function

In [27]:
if __name__ == "__main__":
    main()

/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Training initial model...
Epoch 1/10, Loss: 0.0103
Epoch 2/10, Loss: 0.0087
Epoch 3/10, Loss: 0.0085
Epoch 4/10, Loss: 0.0084
Epoch 5/10, Loss: 0.0084
Epoch 6/10, Loss: 0.0083
Epoch 7/10, Loss: 0.0082
Epoch 8/10, Loss: 0.0081
Epoch 9/10, Loss: 0.0080
Epoch 10/10, Loss: 0.0079
Evaluating initial model...
MSE: 0.0081, MAE: 0.0497, R²: 0.1392
Model saved to ./output/strategies/lstm_initial.pth and ./output/strategies/lstm_initial.onnx

Pruning with strategy: magnitude


/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch_pruning/pruner/algorithms/base_pruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn(
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [25,0,0], 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
